In [ ]:
using Interact
using Plots
using Dates

In [ ]:
function tick_clock(sec)
    rad = sec * 2pi / 60
    r = mod(rad, 2pi)
    r = pi / 2 - r
    # make needle
    q = quiver([0], [0],
              quiver = ([cos(r)], [sin(r)]),
              color = :black,
              linewidth = 4,
              linealpha = 0.5)
    # draw tick
    for k in 0:59
        offset = 15
        t = k - offset
        if mod(k, 5) == 0
            b, e = 0.9, 1.1
            plot!([b * cos(-t * 2pi / 60), e * cos(-t * 2pi / 60)],
                  [b * sin(-t * 2pi / 60), e * sin(-t * 2pi / 60)],
                  color = :black)
            annotate!([(1.2 * cos(-t * 2pi / 60), 1.2 * sin(-t * 2pi / 60), "$k")])
        else
            b, e = 0.95, 1.05
            plot!([b * cos(-t * 2pi / 60), e * cos(-t * 2pi / 60)],
                  [b * sin(-t * 2pi / 60), e * sin(-t * 2pi / 60)],
                  color = :black)
        end
    end
    # adjust style
    plot!(xlim = (-1.1, 1.1), ylim = (-1.1, 1.5),
        aspect_ratio = :equal,
        leg = false,
        showaxis = false,
        grid = false,
        title = "$(sec) [s]")
end

In [ ]:
timestatus = Observable(0)
function run_clock(maxsec)
    init_state = 0
    interval = 1
    count = init_state
    while true
        sleep(interval)
        timestatus[] = count
        count += interval
        if count > maxsec
            break
        end
    end
    # reset
    sleep(interval)
    timestatus[] = init_state
end


In [ ]:
btn₊ = button("+")
btn₋ = button("-")
btn₊10 = button("+10")
btn₋10 = button("-10")
const initval = 5
timer = Interact.@map begin
    &btn₊, &btn₋
    s = initval + 10*&btn₊10 + &btn₊ - &btn₋ - 10*&btn₋10
    # do not allow set `s` to negative value.
    s = max(s, 0)
    Widgets.wdglabel("$(s)")
end

btnstart = button("Start")
function dosomething(t)
    maxtime = parse(Int, t.val.children[1])
    run_clock(maxtime)
end

on(_ -> dosomething(timer), btnstart)

buttons = hbox(btn₋10, btn₋, timer, btn₊, btn₊10, btnstart)
clock = map(sec->tick_clock(sec), timestatus)
ui = vbox(buttons,clock)
display(ui)

In [ ]:
using Blink
w = Window()
s=slider(-1:0.1:1)
body!(w, ui)

In [ ]:
using Mux
port=9000
WebIO.webio_serve(page("/", req -> ui),port)

- open http://localhost:9000